In [1]:
import spacy 
import gzip
import re 
import pandas as pd
from spacy.lang.en import English
from collections import Counter
from itertools import chain
import ahocorasick
import ast
from analyze_m_data import get_occupations_list_en, get_occupations_list
from spacy.lang.de import German
from spacy.lang.it import Italian
from spacy.lang.es import Spanish 
import matplotlib.pyplot as plt
import itertools
from collections import Counter
from operator import itemgetter

In [2]:
filename = "/mount/studenten/projects/caulfiea/cross-verified-database.csv.gz"
csvFilename = gzip.open(filename, 'rb')
df = pd.read_csv(csvFilename, encoding='latin-1')
print(df.head(10))
print(df.columns)

  wikidata_code   birth   death  updated_death_date approx_birth approx_death  \
0      Q1000002  1932.0  1990.0                 NaN          NaN          NaN   
1      Q1000005  1860.0  1927.0                 NaN          NaN          NaN   
2      Q1000006  1971.0     NaN                 NaN          NaN          NaN   
3      Q1000015  1983.0     NaN                 NaN          NaN          NaN   
4      Q1000023  1912.0  1977.0                 NaN          NaN          NaN   
5      Q1000026  1928.0  2016.0                 NaN          NaN          NaN   
6      Q1000034  1818.0  1894.0                 NaN          NaN          NaN   
7      Q1000044  1907.0  1980.0                 NaN          NaN          NaN   
8      Q1000045  1915.0  1987.0                 NaN          NaN        circa   
9      Q1000048  1891.0  1962.0                 NaN          NaN          NaN   

   birth_min  birth_max  death_min  death_max  ... area2_of_rattachment  \
0     1932.0     1932.0     1990.

In [4]:
print(df.columns)

Index(['wikidata_code', 'birth', 'death', 'updated_death_date', 'approx_birth',
       'approx_death', 'birth_min', 'birth_max', 'death_min', 'death_max',
       'gender', 'level1_main_occ', 'name', 'un_subregion', 'birth_estimation',
       'death_estimation', 'bigperiod_birth_graph_b',
       'bigperiod_death_graph_b', 'curid', 'level2_main_occ', 'freq_main_occ',
       'freq_second_occ', 'level2_second_occ', 'level3_main_occ',
       'bigperiod_birth', 'bigperiod_death', 'wiki_readers_2015_2018',
       'non_missing_score', 'total_count_words_b', 'number_wiki_editions',
       'total_noccur_links_b', 'sum_visib_ln_5criteria',
       'ranking_visib_5criteria', 'all_geography_groups',
       'string_citizenship_raw_d', 'citizenship_1_b', 'citizenship_2_b',
       'list_areas_of_rattach', 'area1_of_rattachment', 'area2_of_rattachment',
       'list_wikipedia_editions', 'un_region', 'group_wikipedia_editions',
       'bplo1', 'dplo1', 'bpla1', 'dpla1', 'pantheon_1', 'level3_all_occ'],
 

In [27]:
df_sorted = df.sort_values(by=['number_wiki_editions'],ascending=False)
# df_sorted = df_sorted[df_sorted['list_wikipedia_editions'].isin(['enwiki'])]
df_sorted = df_sorted[df_sorted['list_wikipedia_editions'].str.contains('enwiki')]
df_sorted_female = df_sorted[df_sorted['gender'] == 'Female']
print(df_sorted_female[['name','number_wiki_editions','list_wikipedia_editions']].head(20))
print(df_sorted_female.shape[0])

                           name  number_wiki_editions  \
1391415          Marilyn_Monroe                   168   
2280330            Elizabeth_II                   162   
2060957             Marie_Curie                   159   
2182070           Anna_Netrebko                   158   
616407                Lady_Gaga                   153   
1839188            Hebe_Camargo                   150   
1391016        Marlene_Dietrich                   146   
1390472         Sarah_Bernhardt                   145   
1742419           Angela_Merkel                   145   
746849                   Anggun                   144   
159474                 Agnez_Mo                   141   
1709402             Frida_Kahlo                   139   
2064669             Joan_of_Arc                   135   
2117188       Margaret_Thatcher                   134   
2063392  Elizabeth_I_of_England                   133   
1188610   Mary,_mother_of_Jesus                   131   
1060370           Mother_Teresa

In [28]:
df_sorted = df.sort_values(by=['number_wiki_editions'],ascending=False)
# df_sorted = df_sorted[df_sorted['list_wikipedia_editions'].isin(['enwiki'])]
df_sorted = df_sorted[df_sorted['list_wikipedia_editions'].str.contains('enwiki')]
df_sorted_female = df_sorted[df_sorted['gender'] == 'Male']
print(df_sorted_female[['name','number_wiki_editions','list_wikipedia_editions']].head(20))
print(df_sorted_female.shape[0])

                            name  number_wiki_editions  \
2291069            Ronald_Reagan                   249   
1048985                    Jesus                   245   
962453           Michael_Jackson                   235   
2140022             Barack_Obama                   229   
1391483              Corbin_Bleu                   215   
2146831        Leonardo_da_Vinci                   212   
2265833             Isaac_Newton                   210   
1390405                Confucius                   209   
2212182           Nelson_Mandela                   206   
1201823             Adolf_Hitler                   205   
857902   Wolfgang_Amadeus_Mozart                   203   
2266980          Albert_Einstein                   202   
30172              Julius_Caesar                   201   
758887              Donald_Trump                   198   
1710447             Michelangelo                   193   
2012045      William_Shakespeare                   192   
2236584       

In [30]:
df_german = df[df['list_wikipedia_editions'].str.contains('dewiki')]
df_german_female = df_german[df_german['gender'] == 'Female']
df_german_male = df_german[df_german['gender'] == 'Male']
print(df_german_female.shape[0],df_german_male.shape[0])

108630 590622


In [31]:
df_italian = df[df['list_wikipedia_editions'].str.contains('itwiki')]
df_italian_female = df_italian[df_italian['gender'] == 'Female']
df_italian_male = df_italian[df_italian['gender'] == 'Male']
print(df_italian_female.shape[0],df_italian_male.shape[0])

54755 302621


In [32]:
df_spanish = df[df['list_wikipedia_editions'].str.contains('eswiki')]
df_spanish_female = df_spanish[df_spanish['gender'] == 'Female']
df_spanish_male = df_spanish[df_spanish['gender'] == 'Male']
print(df_spanish_female.shape[0],df_spanish_male.shape[0])

69118 272555


In [ ]:
english_female_df = pd.read_csv('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/sets_for_analysis/en_female_occ_all_sentences_full_data.csv', encoding="utf-8")
en_female_filename = "/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/datasets/final_sets/en_female_dataset.csv"
en_female_df = pd.read_csv(en_female_filename, names= ['wikidata_code', 'title', 'intro', 'gender', 'occupations'],header=0, encoding='utf-8')


In [4]:
en_female_codes = english_female_df['wikidata_code'].to_list()
df2 = df.loc[df['wikidata_code'].isin(en_female_codes)]
print(df2.shape[0])

102444


In [5]:
notability_scores = df2[['wikidata_code','number_wiki_editions','sum_visib_ln_5criteria']]
result = pd.merge(english_female_df, notability_scores, on='wikidata_code')
print(result.head())

   Unnamed: 0 wikidata_code                         title  \
0           0       Q101138           elisabeth bauriedel   
1           1     Q10305226  janet auchincloss rutherfurd   
2           2     Q10320901                 luiza mariani   
3           3     Q10325735         maria baptista soares   
4           4     Q10346103                 patricia seed   

                                               intro  gender  \
0  elisabeth bauriedel (born 20 june 1939) was a ...  female   
1  janet jennings auchincloss rutherfurd (june 13...  female   
2  luiza mariani is a brazilian actress and produ...  female   
3  maria baptista soares (born november 15, 1928)...  female   
4  patricia seed is an american historian and pro...  female   

                           occupations  \
0                       ['politician']   
1                        ['socialite']   
2                            ['actor']   
3                           ['writer']   
4  ['historian', 'university teacher']

In [3]:
en_female_filename = "/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/datasets/redo/en_female_dataset.csv"
en_female_df = pd.read_csv(en_female_filename, names= ['wikidata_code', 'title', 'intro', 'gender', 'occupations'],header=0, encoding='utf-8')
en_female_codes = en_female_df['wikidata_code'].to_list()
df2 = df.loc[df['wikidata_code'].isin(en_female_codes)]
print(df2.shape[0])
notability_scores = df2[['wikidata_code','number_wiki_editions','sum_visib_ln_5criteria']]
result = pd.merge(en_female_df, notability_scores, on='wikidata_code')
result_sorted = result.sort_values(by=['number_wiki_editions'],ascending=False)
print(result_sorted.head(20))

273214
       wikidata_code                  title  \
155625         Q4616         Marilyn Monroe   
272613         Q9682           Elizabeth II   
248478         Q7186            Marie Curie   
263497        Q78516          Anna Netrebko   
56220         Q19848              Lady Gaga   
215939         Q6105           Hebe Camargo   
155494         Q4612       Marlene Dietrich   
155267         Q4605        Sarah Bernhardt   
209020          Q567          Angela Merkel   
71331        Q221310                 Anggun   
13983       Q1362169               Agnez Mo   
205276         Q5588            Frida Kahlo   
248889         Q7226            Joan of Arc   
253837         Q7416      Margaret Thatcher   
248756         Q7207            Elizabeth I   
127945          Q345  Mary, mother of Jesus   
221094         Q6294        Hillary Clinton   
118318        Q30547          Mother Teresa   
267921        Q81731       Murasaki Shikibu   
77718        Q234679     Charlotte Rampling   

     

In [4]:
print("num intros unavailable")
print((result_sorted['intro'] == 'No intro available').sum())

num intros unavailable
7218


In [60]:
print(result.columns)

Index(['Unnamed: 0', 'wikidata_code', 'title', 'intro', 'gender',
       'occupations', 'first_sentence', 'named_entities',
       'overlapping_occupations', 'overlapping_occupations_sentence1',
       'number_wiki_editions', 'sum_visib_ln_5criteria'],
      dtype='object')


In [9]:
result_sorted = result.sort_values(by=['number_wiki_editions'],ascending=False)
print(result_sorted.head(20))

       Unnamed: 0 wikidata_code                   title  \
8261         8603     Q16240162        joelle (actress)   
86037       89505      Q6756367        marcella puppini   
76394       79325      Q6077700            isabel madow   
93007       96801      Q7160734         peggy santiglia   
3877         3979     Q16065733       harriet livermore   
68485       71062     Q54614034           vanessa ponce   
75428       78308     Q57955102         dossibai patell   
75973       78885     Q58823964           irina snopova   
4789         4916     Q16150449  sue smith (politician)   
46297       47941     Q43262624         mikaela mässing   
1691         1756     Q15429774           ellen hollond   
76744       79697      Q6127019          jalynn bennett   
96679      100698      Q7453114        serena dalrymple   
9956        10389     Q16751564            maria rozman   
75543       78431     Q58030786            marie balian   
75925       78830     Q58756444           amelie berger 

In [7]:
def get_split_occupations_en(filename):
    new_df = pd.read_csv(filename, names= ['occupation'],header=0, encoding='utf-8')
    new_df["occupation"] = new_df['occupation'].astype(str)
    # lowercase everything 
    new_df['occupation'] = new_df['occupation'].str.casefold()
    # get rid of extra quotation marks
    new_df['occupation'] = new_df['occupation'].apply(lambda x: x.replace('"', ''))
    # split on commas 
    new_df['occupation'] = new_df['occupation'].apply(lambda x: x.split(','))
    # only take first item in the list split on commas
    new_df['occupation'] = new_df['occupation'].str[0]
    # print(new_df['occupation'].head(20))
    unique_occupations = set(new_df['occupation'].to_list())
    
    # separate strings with and without whitespace
    no_whitespace = [s for s in unique_occupations if " " not in s]
    with_whitespace = [s for s in unique_occupations if " " in s]
    print("All:", len(unique_occupations))
    print("No Whitespace:", len(no_whitespace))
    print("With Whitespace:", len(with_whitespace))
    # print(len(unique_occupations))
    # for i, val in enumerate(itertools.islice(unique_occupations, 10)):
    #     print(val)
    return no_whitespace,with_whitespace

In [45]:
def new_english_analysis(df):
    #Want to: 
    # 1. Split up english occupation titles list into two lists: one token occupations and multi token
    # 2. For one token occupations, want to find matches (in first sentence) with hard word boundaries (like in german code)
    # 3. For multi token occupations, want to just look up matches 
    # 4. Want to remove named entities from matched tokens in that line 
    # 5. Want to get final matched titles list 
    en_1token_occ_titles, en_multitoken_occ_titles = get_split_occupations_en('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/occ_title-lists/eng_gendered_occ_titles.csv')
    print(en_1token_occ_titles)
    df['intro'] = df['intro'].astype(str)
    df['first_sentence'] = df['first_sentence'].astype(str)
    # df['named_entities'] = df['named_entities'].astype(str)
    df['named_entities'] = df['named_entities'].apply(ast.literal_eval)
    # df['named_entities'] = df['named_entities'].str.casefold
    df['named_entities'] = df['named_entities'].apply(lambda x: [item.lower() for item in x])
    print(df['named_entities'].head(10))
    # add wikidata occupations to one token titles
    df['occupations'] = df['occupations'].apply(ast.literal_eval)
    wiki_occupations = set(chain.from_iterable(df['occupations']))
    # en_1token_occ_titles.extend(wiki_occupations)

    # get matching tokens for one word tokens 
    automaton = ahocorasick.Automaton()
    for id, term in enumerate(en_1token_occ_titles):
        automaton.add_word(term, (term))
    automaton.remove_word('-')
    automaton.remove_word('former')
    automaton.remove_word('first')
    automaton.remove_word('second')
    automaton.make_automaton()
    def find_terms_en(text):
        words = list(re.findall(r'\b\w+\b', text.lower()))  # Tokenize text into words
        matches = list(term for _, term in automaton.iter(text) if term.lower() in words)
        return matches
    
    # get matches for multi word tokens (simpler lookup)
    automaton_multi = ahocorasick.Automaton()
    for id, term in enumerate(en_multitoken_occ_titles):
        automaton_multi.add_word(term, (term))
    automaton_multi.remove_word('-')
    automaton_multi.make_automaton()
    def find_terms_en_multi(text):
        return list(term for _, term in automaton_multi.iter(text))
    all_occ_titles = []
    all_occ_titles.extend(en_multitoken_occ_titles)
    all_occ_titles.extend(en_1token_occ_titles)
    automaton_gendered = ahocorasick.Automaton()
    for id, term in enumerate(all_occ_titles):
        term_female= ''.join((' female',' ',term))
        term_male = ''.join((' male',' ',term))
        automaton_gendered.add_word(term_female, (term_female))
        automaton_gendered.add_word(term_male, (term_male))
    automaton_multi.remove_word('-')
    automaton_gendered.make_automaton()
    def find_terms_en_gendered(text):
        return list(term for _, term in automaton_gendered.iter(text))
    
    def find_all_terms(text): 
        single_terms = find_terms_en(text)
        multi_terms = find_terms_en_multi(text)
        single_terms.extend(multi_terms)
        # gendered_terms = find_terms_en_gendered(text)
        # single_terms.extend(gendered_terms)
        return single_terms
    
    def filter_NER(row):
        col1_counts = Counter(row['overlapping_occupations_sentence1'])  # Count occurrences in col1
        col2_counts = Counter(row['named_entities'])  # Count occurrences in col2

        # Remove only up to the number of times they appear in col2
        for word, count in col2_counts.items():
            if word in col1_counts:
                col1_counts[word] -= count  # Reduce count but not below zero
                if col1_counts[word] <= 0:
                    del col1_counts[word]  # Remove word if count reaches zero

        # Reconstruct the filtered list based on updated counts
        filtered_list = []
        for word, count in col1_counts.items():
            filtered_list.extend([word] * count)  # Add back only remaining occurrences

        return filtered_list

    df['overlapping_occupations'] = df['intro'].apply(find_all_terms)
    df['overlapping_occupations_sentence1'] = df['first_sentence'].apply(find_all_terms)
    # remove named entities from matched occupations
    df['overlapping_occupations_sentence1'] = df.apply(filter_NER, axis=1)
    smaller_df = df[df['overlapping_occupations_sentence1'].map(len)>0]
    print("length of dataset:",df.shape[0])
    print("first sentences with occ title found:", smaller_df.shape[0])
    print(df['overlapping_occupations_sentence1'].head(10))
    

In [23]:
new_english_analysis(result)
overlapping_occ_counts_s1_female = Counter(chain.from_iterable(result['overlapping_occupations_sentence1']))
print(overlapping_occ_counts_s1_female)

All: 9284
No Whitespace: 1046
With Whitespace: 8238
['geodesist', 'calibrator', 'baker', 'typist', 'detacker', 'waxer', 'facer', 'milker', 'tapper', 'stand-in', 'engrosser', 'tracer', 'flocker', 'driller-and-reamer', 'opaquer', 'sticker', 'interpreter', 'bouncer', 'loader-unloader', 'pediatrician', 'sounder', 'creaser', 'collector', 'weaver', 'boiler-out', 'tuner', 'zoologist', 'gluer-and-wedger', 'neurologist', 'urologist', 'laborer', 'vulcanizer', 'clerk-of-scales', 'host/hostess', 'dermatologist', 'prefitter', 'sprayer', 'paster', 'stuntwoman', 'pricer', 'cashier-wrapper', 'girl', 'tuckpointer', 'underwriter', 'anatomist', 'compositor', 'postmaster', 'inlayer', 'lacquerer', 'second', 'roller-stitcher', 'critic', 'builder', 'coppersmith', 'vendor', 'photogrammetrist', 'guard', 'screener-perfumer', 'pantographer', 'icer', 'reporter', 'caster', 'bartender', 'assayer', 'lyricist', 'stringer', 'nurse-midwife', 'bricklayer', 'waitress', 'tanker', 'mechanic', 'stratigrapher', 'bender', 'ho

In [33]:
from collections import defaultdict
# Initialize counters
tag_counts = defaultdict(int)
tag_score_totals = defaultdict(float)
tag_counts2 = defaultdict(int)
tag_score_totals2 = defaultdict(float)

# Iterate over rows
for tags, score in zip(result['overlapping_occupations_sentence1'], result['number_wiki_editions']):
    for tag in tags:
        tag_counts[tag] += 1
        tag_score_totals[tag] += score

# Now compute averages
tag_avg_scores = {
    tag: tag_score_totals[tag] / tag_counts[tag]
    for tag in tag_counts
}
for tags2, score in zip(result['overlapping_occupations_sentence1'], result['sum_visib_ln_5criteria']):
    for tag2 in tags2:
        tag_counts2[tag2] += 1
        tag_score_totals2[tag2] += score

# Now compute averages
tag_avg_scores2 = {
    tag2: tag_score_totals2[tag2] / tag_counts2[tag2]
    for tag2 in tag_counts2
}
# Combine into a DataFrame if you want
result_df = pd.DataFrame({
    'count_num_wiki_editions': tag_counts,
    'average_num_wiki_editions': tag_avg_scores,
    'count_sum_visibility': tag_counts2,
    'average_sum_visibility': tag_avg_scores2
})

print(result_df)
print(result_df['average_num_wiki_editions'].mean())
print(result_df['average_sum_visibility'].mean())

                                 count_num_wiki_editions  \
female volleyball player                             375   
female speaker                                         3   
female mayor                                          33   
female bodybuilder                                    74   
female boxer                                          39   
...                                                  ...   
female chef                                            1   
female sports journalist                               1   
female human rights activist                           1   
female film director                                   1   
female para table tennis player                        1   

                                 average_num_wiki_editions  \
female volleyball player                          1.000000   
female speaker                                    1.000000   
female mayor                                      1.000000   
female bodybuilder             

In [30]:
from collections import defaultdict
# Initialize counters
tag_counts = defaultdict(int)
tag_score_totals = defaultdict(float)

# Iterate over rows
for tags, score in zip(result['overlapping_occupations_sentence1'], result['sum_visib_ln_5criteria']):
    for tag in tags:
        tag_counts[tag] += 1
        tag_score_totals[tag] += score

# Now compute averages
tag_avg_scores = {
    tag: tag_score_totals[tag] / tag_counts[tag]
    for tag in tag_counts
}

# Combine into a DataFrame if you want
result_df = pd.DataFrame({
    'count': tag_counts,
    'average_score': tag_avg_scores
})

print(result_df)
print(result_df['average_score'].mean())

                                 count  average_score
female volleyball player           375      14.622757
female speaker                       3      18.747133
female mayor                        33      16.181068
female bodybuilder                  74      18.093353
female boxer                        39      16.534863
...                                ...            ...
female chef                          1      21.754179
female sports journalist             1      15.600292
female human rights activist         1      17.892097
female film director                 1      16.228531
female para table tennis player      1      15.436712

[302 rows x 2 columns]
16.33628008389357


In [50]:
english_male_df = pd.read_csv('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/sets_for_analysis/en_male_occ_all_sentences_full_data.csv', encoding="utf-8")
en_male_codes = english_male_df['wikidata_code'].to_list()
df3 = df.loc[df['wikidata_code'].isin(en_male_codes)]
print(df3.shape[0])
notability_scores = df3[['wikidata_code','number_wiki_editions','sum_visib_ln_5criteria','wiki_readers_2015_2018','total_noccur_links_b','non_missing_score']]
result_male = pd.merge(english_male_df, notability_scores, on='wikidata_code')
print(result_male.head())

94720
   Unnamed: 0 wikidata_code                                 title  \
0           0     Q25999686                       moisés hipólito   
1           1     Q29021452                        frank m. riley   
2           2     Q46916921                 baddam narsimha reddy   
3           3     Q27924891  thomas perry (australian politician)   
4           4      Q5335262                           ed pearlman   

                                               intro gender  \
0  moisés hipólito león (born 15 march 1992) is a...   male   
1  frank m. riley (september 10, 1875 – may 22, 1...   male   
2  baddam narsimha reddy (21 june 1931 – 6 novemb...   male   
3  thomas oswald perry (17 january 1914 – 8 march...   male   
4  ed pearlman is a co-founder (together with don...   male   

                       occupations  \
0  ['association football player']   
1                    ['architect']   
2                   ['politician']   
3                   ['politician']   
4         

In [26]:
new_english_analysis(result_male)
overlapping_occ_counts_s1_male = Counter(chain.from_iterable(result_male['overlapping_occupations_sentence1']))
print(overlapping_occ_counts_s1_male)

All: 9284
No Whitespace: 1046
With Whitespace: 8238
['geodesist', 'calibrator', 'baker', 'typist', 'detacker', 'waxer', 'facer', 'milker', 'tapper', 'stand-in', 'engrosser', 'tracer', 'flocker', 'driller-and-reamer', 'opaquer', 'sticker', 'interpreter', 'bouncer', 'loader-unloader', 'pediatrician', 'sounder', 'creaser', 'collector', 'weaver', 'boiler-out', 'tuner', 'zoologist', 'gluer-and-wedger', 'neurologist', 'urologist', 'laborer', 'vulcanizer', 'clerk-of-scales', 'host/hostess', 'dermatologist', 'prefitter', 'sprayer', 'paster', 'stuntwoman', 'pricer', 'cashier-wrapper', 'girl', 'tuckpointer', 'underwriter', 'anatomist', 'compositor', 'postmaster', 'inlayer', 'lacquerer', 'second', 'roller-stitcher', 'critic', 'builder', 'coppersmith', 'vendor', 'photogrammetrist', 'guard', 'screener-perfumer', 'pantographer', 'icer', 'reporter', 'caster', 'bartender', 'assayer', 'lyricist', 'stringer', 'nurse-midwife', 'bricklayer', 'waitress', 'tanker', 'mechanic', 'stratigrapher', 'bender', 'ho

In [34]:
from collections import defaultdict
# Initialize counters
tag_counts = defaultdict(int)
tag_score_totals = defaultdict(float)
tag_counts2 = defaultdict(int)
tag_score_totals2 = defaultdict(float)

# Iterate over rows
for tags, score in zip(result_male['overlapping_occupations_sentence1'], result['number_wiki_editions']):
    for tag in tags:
        tag_counts[tag] += 1
        tag_score_totals[tag] += score

# Now compute averages
tag_avg_scores = {
    tag: tag_score_totals[tag] / tag_counts[tag]
    for tag in tag_counts
}
for tags2, score in zip(result_male['overlapping_occupations_sentence1'], result['sum_visib_ln_5criteria']):
    for tag2 in tags2:
        tag_counts2[tag2] += 1
        tag_score_totals2[tag2] += score

# Now compute averages
tag_avg_scores2 = {
    tag2: tag_score_totals2[tag2] / tag_counts2[tag2]
    for tag2 in tag_counts2
}
# Combine into a DataFrame if you want
result_df = pd.DataFrame({
    'count_num_wiki_editions': tag_counts,
    'average_num_wiki_editions': tag_avg_scores,
    'count_sum_visibility': tag_counts2,
    'average_sum_visibility': tag_avg_scores2
})

print(result_df)
print(result_df['average_num_wiki_editions'].mean())
print(result_df['average_sum_visibility'].mean())

                              count_num_wiki_editions  \
male volleyball player                             58   
male badminton player                               6   
male water polo player                              6   
male former                                        13   
male weightlifter                                  25   
male long-distance runner                          10   
male page                                           1   
male handball player                                3   
male hammer thrower                                 7   
male acrobat                                       12   
male acrobatic gymnast                             12   
male karateka                                       2   
male model                                          5   
male track cyclist                                  5   
male actor                                          3   
male alpine skier                                   1   
male retired                   

In [55]:
import numpy as np
tag_scores = defaultdict(list)

# Collect all scores per tag
for tags, score in zip(result_male['overlapping_occupations_sentence1'], result_male['non_missing_score']):
    for tag in tags:
        tag_scores[tag].append(score)

# Compute statistics
tag_stats = {
    tag: {
        'count': len(scores),
        'average_score': np.mean(scores),
        'median_score': np.median(scores),
        'min_score': np.min(scores),
        'max_score': np.max(scores)
    }
    for tag, scores in tag_scores.items()
}

# Create DataFrame and sort
result_df = pd.DataFrame.from_dict(tag_stats, orient='index')
result_df = result_df.sort_values(by='average_score', ascending=False)

print(result_df)

                    count  average_score  median_score  min_score  max_score
crew member             3       3.000000           3.0          3          3
fireman                10       3.000000           3.0          3          3
headmistress            1       3.000000           3.0          3          3
chairwoman              1       3.000000           3.0          3          3
hostess                 1       3.000000           3.0          3          3
bartender               1       3.000000           3.0          3          3
mail carrier            1       3.000000           3.0          3          3
headteacher             1       3.000000           3.0          3          3
server                  1       3.000000           3.0          3          3
weatherman              4       3.000000           3.0          3          3
mailman                 1       3.000000           3.0          3          3
weather forecaster      8       3.000000           3.0          3          3

In [49]:
new_english_analysis(result)
overlapping_occ_counts_s1_female = Counter(chain.from_iterable(result['overlapping_occupations_sentence1']))
print(overlapping_occ_counts_s1_female)

All: 60
No Whitespace: 52
With Whitespace: 8
['saleswoman', 'crewman', 'fireman', 'hostess', 'stuntwoman', 'actor', 'comedienne', 'weathergirl', 'sewist', 'mailwoman', 'comedian', 'meteorologist', 'policeman', 'landlady', 'seamstress', 'salesman', 'businesswoman', 'stuntman', 'host', 'businessman', 'headmistress', 'clergywoman', 'camerawoman', 'stuntperson', 'salesperson', 'policewoman', 'chairman', 'headmaster', 'firefighter', 'landlord', 'chairperson/chair', 'server', 'anchorwoman', 'mailman', 'bartender', 'anchorman', 'steward', 'headteacher', 'clergyman', 'barman', 'stewardess', 'crewwoman', 'businessperson', 'chairwoman', 'barkeeper', 'minister/pastor', 'waiter', 'cameraman', 'waitress', 'actress', 'barmaid', 'weatherman']
0    [elisabeth, bauriedel, 20, june, 1939, german,...
1    [janet, jennings, auchincloss, rutherfurd, jun...
2    [luiza, mariani, brazilian, 14, years, old, 15...
3    [maria, baptista, soares, november, 15, ,, 192...
4    [patricia, seed, american, the, unive

In [51]:
new_english_analysis(result_male)
overlapping_occ_counts_s1_male = Counter(chain.from_iterable(result_male['overlapping_occupations_sentence1']))
print(overlapping_occ_counts_s1_male)

All: 60
No Whitespace: 52
With Whitespace: 8
['saleswoman', 'crewman', 'fireman', 'hostess', 'stuntwoman', 'actor', 'comedienne', 'weathergirl', 'sewist', 'mailwoman', 'comedian', 'meteorologist', 'policeman', 'landlady', 'seamstress', 'salesman', 'businesswoman', 'stuntman', 'host', 'businessman', 'headmistress', 'clergywoman', 'camerawoman', 'stuntperson', 'salesperson', 'policewoman', 'chairman', 'headmaster', 'firefighter', 'landlord', 'chairperson/chair', 'server', 'anchorwoman', 'mailman', 'bartender', 'anchorman', 'steward', 'headteacher', 'clergyman', 'barman', 'stewardess', 'crewwoman', 'businessperson', 'chairwoman', 'barkeeper', 'minister/pastor', 'waiter', 'cameraman', 'waitress', 'actress', 'barmaid', 'weatherman']
0    [hipólito, león, 15, march, 1992, mexican, cru...
1    [frank, m., riley, september, 10, ,, 1875, may...
2    [21, june, 1931, november, 2017, indian, the, ...
3    [thomas, oswald, perry, 17, january, 1914, mar...
4    [ed, pearlman, don, francisco, the, n